In [1]:
import pandas as pd
from json import load

df_geo = pd.read_csv('./data/geolocalizacion.csv', sep=';')
df_car = pd.read_csv('./data/caracteristicas.csv', sep=';')
df_web = pd.read_csv('./data/web.csv', sep=';')

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
df_meta_geo = df_geo[['rk', 'latitud', 'longitud', 'codigo_postal', 'pais', 'provincia', 'ciudad', 'distrito', 'barrio', 'calle', 'planta']]
"""
Desechamos las columnas de metadatos que no van a influir en nuestro knn. 
Esta información, en el caso de llegar a representarla lo que haremos será enriquecer con una api en una tabla exclusiva para ello 
ennuestra BBD


necesario:  
            [rk] necesitamos el identificador del piso.
            [planta] es un valor muy necesario para nuestro  algoritmo

útilies:   
            [geo] no es necesario si los barrios estuviesen bien definidios. pero como aún no tenemos clara la manera de adquirir el barrio lo mantenemos.
            [CP] si viéramos que los barrios son una lista de codigos postales podríamos ahorrarnos el uso del shapefile y simplemente relacionar CP con barrios. 
            [Pais] provincia y ciudad podrían ser útiles si están bien metidos los datos.
            [distrito y barrio] están un poco sucios pero los mantenemmos aunque luego lo machaquemos con nuestraa información.df_geo
            [calle] podría no ser necesario para nuestro algoritmo pero por ahora lo mantenemos, seguramente lo desechemos más adelante.
"""
df_meta_geo.head(3)


,rk,latitud,longitud,codigo_postal,pais,provincia,ciudad,distrito,barrio,calle,planta
0,UV/2020/000001,403.858,-372.599,28019,1,1,Madrid,Carabanchel,NaN,Avenida de Oporto,1
1,UV/2020/000003,404.278,-363.083,28037,1,1,Madrid,San Blas,Canillejas,Avenida Institucion Libre de Enseñanza,9
2,UV/2020/000004,401.924,-366.922,28341,1,1,Valdemoro,Zona estación,NaN,Calle Tenerife,1


In [3]:
df_meta_car=df_car[['uv', 'estado', 'habitaciones', 'banos','metros','fecha_construccion','orientacion','situacion', 'portal', 'ascensor', 'parque_infantil', 'terraza', 'trastero', 'piscina_comunitaria', 'cuarto_de_basura','zonas_comunes', 'piscina', 'garaje', 'tejado', 'calefaccion', 'jardin', 'aire_acondicionado', 'seguro_hogar']]
"""
Desechamos las columnas de datos que no crremos que nos vayan a influir.complex

eliminamos: 
            [rk] no necesitamos saber en una misma columna el estado del piso y su identificador. lo tenemos cada uno en su columna correspondiente. Posteriormente filtraremos aquellos pisos repetidos que por haber sido reformados nos siguen apareciendo sin reformar.Además tendremos que ser consistenetes con la nomenclartura de esta columna ya que rk de la tabla de geo y uv de esta tabla contienen la misma información, no es así con rk de esta tabla.complex

            [metrica] es inconsistente que tengamos una columna "metros" y redundemos en que la métrica es m2. Asumimos que todos los inmuebles están en metros y que si fuese necesario incluir en medidas imperiales (por ejemplo) se transformaría a metros o se transformaría a metros y se incluría una columna con la información en yardas en todas las viviendas.complex

            [certificaciones] [certificado_energetico]
            

mantenemos: 
            Muy importante para nuestro algoritmo
            [ascensor ]

            Podría ser interesante para nuestro algoritmo 
            [fecha_construccion] [terraza] [orientacion] [situacion][portal][parque_infantil] [trastero] [piscina_comunitaria] [piscina] [cuarto_de_basura] [zonas_comunes] [garaje] [tejado] [calefaccion]  [jardin] [aire_acondicionado] [seguro_hogar]
            

Mantenemos muchas columnas poco relevantes pero el poderador les dará su valor.

NOTA: columna  [portal] es si tiene o no portero automático. columana [techo] es si está o no abuardillado 

"""

#puede que algunos estados no sean ni '1' ni '2', consideramos 'otro' como la única opción

df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '?'


#tenemos algunos valores repetidos como:
df_meta_car[df_meta_car.uv == 'UV/2020/000011']

<ipython-input-3-926bd20fa7e5>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '?'
/home/emilio/.local/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


,uv,estado,habitaciones,banos,metros,fecha_construccion,orientacion,situacion,portal,ascensor,...,piscina_comunitaria,cuarto_de_basura,zonas_comunes,piscina,garaje,tejado,calefaccion,jardin,aire_acondicionado,seguro_hogar
9,UV/2020/000011,1,3,1,83,1966.0,Suroeste,Exterior,0,0,...,0,0,0,0,0,0.0,0,0,0,0
10,UV/2020/000011,2,3,1,83,1966.0,Suroeste,NaN,1,1,...,0,0,0,0,0,0.0,0,0,0,0


In [4]:
df_meta_car.tejado.value_counts()

0.0    398
Name: tejado, dtype: int64

In [5]:
df_meta_web=df_web[['uv', 'estado', 'tipo_vivienda', 'precio', 'gastos_comunitarios']]
"""
Mantenemos prácticamente todos los datos.

Alunos comentarios:
    [estado] debería ser redundante ya que lo tenemos tb en df_meta_car pero nos servirá para asegurarnos de que el dato es correcto
    [precio_antiguo] [descuento] los desechamos, nuestro algoritmo sólo considerará el precio actual. si es con descuento o no será el comercial quien tenga esta información y recalcará al clliente que se estaría aprovechando de un descuento.
    [gastos_comunitarios] esta variable podría ser interesante para nuestro algoritmo así que la mantenemos. en el momento de recomendar una u otra casa puede que sí sea información valiosa para los comerciales: nos convendría vender antes un piso con una cuota de gastos comunitarios alta que una con pocos gastos comunitarios.
    
    eliminanos
    [moneda] suponemos todos los precios en euros
"""
df_meta_web.sample(3)

,uv,estado,tipo_vivienda,precio,gastos_comunitarios
56,UV/2020/000213,1,Piso,165000.0,0.0
71,UV/2020/000249,1,Piso,169000.0,0.0
46,UV/2020/000151,1,Piso,360000.0,0.0


In [6]:
df_meta_geo.columns=['uv', 'latitud', 'longitud', 'codigo_postal', 'pais', 'provincia',
       'ciudad', 'distrito', 'barrio', 'calle', 'planta']
#mantenemmos consistente nuestra nomenclatura del identificador y llamamos a todos los uv correctamente (no rk)


In [7]:
"""
 En primer lugar deberemos eliminar las viviendas que, habiendo sido reformadas, nos osiguen apareciendo con la opción sin reformar
"""

#2ddo esta función elimina los duplicados pero se queda con el que tenga el estado más alto y podrían perderse datos. 

df_meta_car = df_meta_car.sort_values('estado').drop_duplicates('uv',keep='last')
###Exixten viviendas repetidas y algunas de ellas tienen valores inconsistentes, por lo tanto vamos a intentar unificar.


In [8]:
def saca_NS(cadena):
    CADENA=cadena.upper()
    if 'NOR' in CADENA: return 'N'
    elif 'SU' in CADENA: return 'S'
    else: return ''

def saca_EO(cadena):
    CADENA=cadena.upper()
    if 'OES' in CADENA: return 'O'
    elif 'EST' in CADENA: return 'E'
    else: return ''

In [9]:
df_meta_car['NS']=df_meta_car.orientacion
df_meta_car['EO']=df_meta_car.orientacion

df_meta_car['NS']=df_meta_car['orientacion'].apply(saca_NS)
df_meta_car['EO']=df_meta_car['orientacion'].apply(saca_EO)
df_meta_car.drop(['orientacion'], axis=1, inplace= True)

In [10]:

#solo permitimos el estado '1' o '2', si no marcamos'3'
df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '3'


<ipython-input-10-846d4d538a85>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_car.estado[(df_meta_car.estado != '2') & (df_meta_car.estado != '1')] = '3'


In [11]:
df_meta_car.tejado.value_counts()

0.0    370
Name: tejado, dtype: int64

In [12]:
df_meta_car.situacion.value_counts()

Exterior            307
Interior             46
Patio de manzana     11
Name: situacion, dtype: int64

portal                 0
ascensor               0
parque_infantil        0
terraza                0
trastero               0
piscina_comunitaria    0
cuarto_de_basura       0
zonas_comunes          0
piscina                0
garaje                 0
tejado                 0
calefaccion            0
jardin                 0
aire_acondicionado     0
dtype: int64

In [13]:
df_meta_car[[ 'estado', 'habitaciones', 'banos', 'metros','fecha_construccion', 'tejado']] = df_meta_car[[ 'estado', 'habitaciones', 'banos', 'metros','fecha_construccion', 'tejado']].fillna(0.0).astype(int)
df_meta_car.situacion=df_meta_car.situacion.fillna('NO').map({'Exterior':int(3),"Patio de manzana":int(2),"Interior":int(1), 'NO':int(0)}).astype(int)
df_meta_car[['portal', 'ascensor','parque_infantil', 'terraza', 'trastero', 'piscina_comunitaria', 'cuarto_de_basura', 'zonas_comunes', 'piscina', 'garaje', 'tejado','calefaccion','jardin', 'aire_acondicionado']] = df_meta_car[['portal', 'ascensor','parque_infantil', 'terraza', 'trastero', 'piscina_comunitaria', 'cuarto_de_basura', 'zonas_comunes', 'piscina', 'garaje', 'tejado','calefaccion','jardin', 'aire_acondicionado']].fillna(0).astype(int)

In [14]:
df_meta_car

,uv,estado,habitaciones,banos,metros,fecha_construccion,situacion,portal,ascensor,parque_infantil,...,zonas_comunes,piscina,garaje,tejado,calefaccion,jardin,aire_acondicionado,seguro_hogar,NS,EO
0,UV/2020/000001,1,2,1,53,1963,3,0,1,0,...,0,0,0,0,0,0,0,0,N,
252,UV/2020/000301,1,3,2,80,1983,3,0,0,0,...,0,0,0,0,0,0,0,0,N,O
251,UV/2020/000300,1,4,1,120,1965,1,0,1,0,...,0,0,0,0,0,0,0,0,N,O
250,UV/2020/000299,1,2,2,101,1940,3,1,1,0,...,0,0,0,0,0,0,0,0,N,E
249,UV/2020/000297,1,1,1,59,2011,3,1,1,0,...,1,0,1,0,0,0,0,0,N,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,UV/2020/000307,2,2,2,70,1950,3,0,1,0,...,0,0,0,0,0,0,0,0,,E
266,UV/2020/000309,2,3,1,65,1963,1,0,0,0,...,0,0,0,0,0,0,0,0,,O
361,UV/2021/000397,2,2,2,73,1973,3,1,1,0,...,0,0,0,0,0,0,0,0,S,O
293,UV/2020/000330,2,2,1,41,1914,1,1,0,0,...,0,0,0,0,0,0,0,0,N,E


<ipython-input-15-d8f42811b9ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_web.tipo_vivienda[df_meta_web.tipo_vivienda == 'Moradia']= 'Casa'
/home/emilio/.local/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [15]:
df_meta_web.tipo_vivienda[df_meta_web.tipo_vivienda == 'Moradia'] = 'CASA'

def filtrar_tipo_vivienda(cadena):
    with open("./config/opciones.json") as archivo:
        opciones = load(archivo)
    conjunto_vivientas = set(opciones['conjunto_vivientas'])
    CADENA = cadena.upper()
    if CADENA in conjunto_vivientas: return CADENA
    else: return 'OTRO'


<ipython-input-15-3717ac23ec7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_web.tipo_vivienda[df_meta_web.tipo_vivienda == 'Moradia'] = 'CASA'
/home/emilio/.local/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [16]:
df_meta_web.tipo_vivienda = df_meta_web.tipo_vivienda.apply(filtrar_tipo_vivienda)

/home/emilio/.local/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
df_meta_web.precio =  df_meta_web.precio.fillna(0).astype(int)

In [18]:
df_meta_web.gastos_comunitarios =  df_meta_web.gastos_comunitarios.fillna(0).astype(int)

In [19]:
df_meta_web.precio =  (df_meta_web.precio/1000).astype(int)

In [20]:
df_meta_web.columns=['uv', 'estado', 'tipo_vivienda', 'precio_k', 'gastos_comunitarios']
df_meta_web

,uv,estado,tipo_vivienda,precio_k,gastos_comunitarios
0,UV/2020/000003,1,PISO,160,0
1,UV/2020/000004,1,PISO,125,0
2,UV/2020/000006,1,PISO,0,0
3,UV/2020/000007,1,PISO,159,0
4,UV/2020/000008,1,PISO,170,0
...,...,...,...,...,...
185,UV/2021/000392,1,PISO,139,18
186,UV/2021/000411,1,PISO,235,140
187,UV/2021/000426,1,PISO,148,64
188,UV/2021/000433,1,PISO,300,30


In [21]:
df_meta_web[df_meta_web.uv == 'UV/2021/000452']

,uv,estado,tipo_vivienda,precio_k,gastos_comunitarios
153,UV/2021/000452,1,CASA,0,12
154,UV/2021/000452,1,CASA,0,12
155,UV/2021/000452,2,CASA,0,12
156,UV/2021/000452,2,CASA,0,12
